#  加油站前端数据分析--用户画像

In [1]:
from pylab import *
import pandas as pd
import scipy as sp
#import pyecharts as es

### 1、载入加油站的三个月本地数据
载入加油站三个月本地到pandas的dataframe

In [2]:
import sqlite3
conn = sqlite3.connect('xbxIOTGatherAppMainV3.db')
sql = 'select * from tbIOTOilGatherMainTable '
data = pd.read_sql(sql, conn)
conn.close()

In [3]:
data.head()

,JYLX,GYRXM,CYZJDM,ZJHM,XBDM,MZDM,DZMC,JDCHPHM,YPLX,GYL,...,ZMCL5,ZMCL6,ZMCL7,ZMCL8,YPDJ,BDJG,JDCCLLXDM,JDCHPYS,iState,JYSJ
0,02,吐尔逊·麦麦提,111,652901198106032815,男,维吾尔,新疆阿克苏市南城街道南郊路7号2巷45号,新N3Q850,12,5.26,...,,,,,5.7,0.909483,20,1,4,2017-08-27T16:44:37.803
1,01,玉苏甫江·麦提尼亚孜,111,652901199106073216,男,维吾尔,新疆阿克苏市拜什吐格曼乡玉素甫霍加艾日克村4组38号,新N8E045,12,6.06,...,,,,,6.6,0.889286,0,0,4,2018-02-19T11:56:08.313
2,02,周宏顺,111,432929196302070516,男,汉,湖南省双牌县五里牌镇线口村3组,新NK5565,12,7.69,...,,,,,6.5,0.688988,20,1,6,2018-03-28T13:22:48.991
3,01,童丽,111,652901197905080026,女,汉,新疆阿克苏市新华东路100号,新NHD669,12,15.38,...,,,,,6.5,0.803510,0,0,6,2018-03-28T13:22:58.071
4,01,陈学长,111,411024197302111671,男,汉,河南省鄢陵县张桥乡大路袁村９１号,豫KYZ209,12,3.08,...,,,,,6.5,0.707631,0,0,6,2018-03-28T13:23:07.535


In [4]:
data.columns

Index(['JYLX', 'GYRXM', 'CYZJDM', 'ZJHM', 'XBDM', 'MZDM', 'DZMC', 'JDCHPHM',
       'YPLX', 'GYL', 'JYY', 'YHBZ', 'CJSJ', 'ZJZP', 'CLZP', 'GYRYXCZP',
       'ZMCL1', 'ZMCL2', 'ZMCL3', 'ZMCL4', 'ZMCL5', 'ZMCL6', 'ZMCL7', 'ZMCL8',
       'YPDJ', 'BDJG', 'JDCCLLXDM', 'JDCHPYS', 'iState', 'JYSJ'],
      dtype='object')

In [5]:
data.iloc[0]

JYLX                                                        02
GYRXM                                                  吐尔逊·麦麦提
CYZJDM                                                     111
ZJHM                                        652901198106032815
XBDM                                                         男
MZDM                                                       维吾尔
DZMC                                      新疆阿克苏市南城街道南郊路7号2巷45号
JDCHPHM                                                新N3Q850
YPLX                                                        12
GYL                                                       5.26
JYY                                                        武建军
YHBZ                                               阿克苏金盛加油站管理员
CJSJ                                   2017-08-27T16:41:58.653
ZJZP                                img/652901198106032815.jpg
CLZP                              plates/hc_453_1503823319.jpg
GYRYXCZP     D:/OilInfoGatherClient/faces/reg_652901198

In [6]:
data['JYLX'].unique()

array(['02', '01', '00'], dtype=object)

In [7]:
data.shape

(37022, 30)

In [8]:
data['JYSJ'] = pd.to_datetime(data['JYSJ'],format='%Y-%m-%dT%H:%M:%S.%f')

In [9]:
data['JYSJ'].head()

0   2017-08-27 16:44:37.803
1   2018-02-19 11:56:08.313
2   2018-03-28 13:22:48.991
3   2018-03-28 13:22:58.071
4   2018-03-28 13:23:07.535
Name: JYSJ, dtype: datetime64[ns]

In [10]:
data1=data.set_index(['JYSJ'])

In [11]:
data1.sort_index(inplace=True)

In [12]:
data = data1[data1.index >'20180327']

### 2、对加油站数据进行预测
载入加油站三个月本地到pandas的dataframe
最终生成的DataFrame是包含三个月完整数据的；可以用来分星期进行加油数据的预测

In [13]:
data.shape

(37020, 29)

可以看到数据维度没有问题，然后我们需要对这些数据进行处理，以便于得到每个星期的加油总量，目前能够想到的待提取特征如下：
加油人数、加油车辆数、加油摩托数、散装油数、车辆加油量、摩托加油量、散装加油量、--与上一星期的差值--、--与上上一星期的差值--
上述特征对应：下一星期的总加油量


##### 2.1将所有加油记录按照星期进行组织

In [46]:
dataFrom = data

In [63]:
newData = dataFrom.index.to_period("W")

In [64]:
dataFrom1 = dataFrom.set_index(newData)

In [65]:
dataFrom1

,JYLX,GYRXM,CYZJDM,ZJHM,XBDM,MZDM,DZMC,JDCHPHM,YPLX,GYL,...,ZMCL4,ZMCL5,ZMCL6,ZMCL7,ZMCL8,YPDJ,BDJG,JDCCLLXDM,JDCHPYS,iState
JYSJ,,,,,,,,,,,,,,,,,,,,,
2018-03-26/2018-04-01,01,吴超,111,411324199009170037,男,回,河南省镇平县城关镇北关村１１组６１号,新NW9919,12,7.690000,...,,,,,,6.5,0.830714,0,0,6
2018-03-26/2018-04-01,01,陈江华,111,65290119931228281X,男,土家,新疆阿克苏市良种场场部177号,新N7X365,12,7.690000,...,,,,,,6.5,0.787546,0,0,6
2018-03-26/2018-04-01,02,周宏顺,111,432929196302070516,男,汉,湖南省双牌县五里牌镇线口村3组,新NK5565,12,7.690000,...,,,,,,6.5,0.688988,20,1,6
2018-03-26/2018-04-01,01,童丽,111,652901197905080026,女,汉,新疆阿克苏市新华东路100号,新NHD669,12,15.380000,...,,,,,,6.5,0.803510,0,0,6
2018-03-26/2018-04-01,01,陈学长,111,411024197302111671,男,汉,河南省鄢陵县张桥乡大路袁村９１号,豫KYZ209,12,3.080000,...,,,,,,6.5,0.707631,0,0,6
2018-03-26/2018-04-01,01,吐尔洪·达吾提,111,652901198510231138,男,维吾尔,新疆阿克苏市友谊路28号7栋8号,新N2T955,12,7.690000,...,,,,,,6.5,0.731781,0,0,6
2018-03-26/2018-04-01,02,托合提·买米提明,111,652901195210103616,男,维吾尔,新疆阿克苏市多浪路149号,新NF1795,12,3.080000,...,,,,,,6.5,0.877648,20,1,6
2018-03-26/2018-04-01,01,努尔加马力·沙的克,111,652901197401241121,女,维吾尔,新疆阿克苏市环南路36号1号楼3单元402室,新N7G329,12,15.380000,...,,,,,,6.5,0.844163,0,0,6
2018-03-26/2018-04-01,01,陈学长,111,411024197302111671,男,汉,河南省鄢陵县张桥乡大路袁村９１号,豫KY7209,12,7.690000,...,,,,,,6.5,0.776181,0,0,6


In [92]:
mm=dataFrom1.groupby(['JYSJ', 'JYLX']).agg({'GYL':['count','sum']})

In [93]:
mm

GYL               
                           count            sum
JYSJ                  JYLX                     
2018-03-26/2018-04-01 00      58     489.169989
                      01    1463   19053.770096
                      02     347  232700.369982
2018-04-02/2018-04-08 00      86     328.829997
                      01    2161   31590.070201
                      02     508    1547.839968
2018-04-09/2018-04-15 00      57     399.789993
                      01    2255   28813.460168
                      02     464    1724.629969
2018-04-16/2018-04-22 00      72     435.190000
                      01    2464   31236.690169
                      02     587    2235.309954
2018-04-23/2018-04-29 00      75     588.699989
                      01    2263   30524.640181
                      02     577    2307.659959
2018-04-30/2018-05-06 00      75     534.929994
                      01    2210   31293.460190
                      02     610    2417.669950
2018-05-07/2018-05-13 00      54     417.159994
                      01    2276   29550.990182
                      02     649    2637.859953
2018-05-14/2018-05-20 00      54     242.640000
                      01    2170   31583.730082
                      02     604    2136.149990
2018-05-21/2018-05-27 00      80     551.200005
                      01    2196   28026.390288
                      02     622    2374.040017
2018-05-28/2018-06-03 00      76     373.400001
                      01    2180   29973.760466
                      02     626    2192.470004
2018-06-04/2018-06-10 00      67     547.559999
                      01    2143   27855.620429
                      02     641    2449.940010
2018-06-11/2018-06-17 00     100     636.090000
                      01    2120   30393.230500
                      02     548    1971.470007
2018-06-18/2018-06-24 00     124     961.240000
                      01    2146   28093.310436
                      02     619    2401.470007
2018-06-25/2018-07-01 00      22     148.050000
                      01     445    5864.300093
                      02     126     500.660001

In [98]:
mm1=dataFrom1[[ 'JYLX','GYL']].groupby(['JYSJ', 'JYLX']).apply(sum)

In [99]:
mm1

JYLX            GYL
JYSJ                  JYLX                              
2018-03-26/2018-04-01 00     0.000000e+00     489.169989
                      01              inf   19053.770096
                      02              inf  232700.369982
2018-04-02/2018-04-08 00     0.000000e+00     328.829997
                      01              inf   31590.070201
                      02              inf    1547.839968
2018-04-09/2018-04-15 00     0.000000e+00     399.789993
                      01              inf   28813.460168
                      02              inf    1724.629969
2018-04-16/2018-04-22 00     0.000000e+00     435.190000
                      01              inf   31236.690169
                      02              inf    2235.309954
2018-04-23/2018-04-29 00     0.000000e+00     588.699989
                      01              inf   30524.640181
                      02              inf    2307.659959
2018-04-30/2018-05-06 00     0.000000e+00     534.929994
                      01              inf   31293.460190
                      02              inf    2417.669950
2018-05-07/2018-05-13 00     0.000000e+00     417.159994
                      01              inf   29550.990182
                      02              inf    2637.859953
2018-05-14/2018-05-20 00     0.000000e+00     242.640000
                      01              inf   31583.730082
                      02              inf    2136.149990
2018-05-21/2018-05-27 00     0.000000e+00     551.200005
                      01              inf   28026.390288
                      02              inf    2374.040017
2018-05-28/2018-06-03 00     0.000000e+00     373.400001
                      01              inf   29973.760466
                      02              inf    2192.470004
2018-06-04/2018-06-10 00     0.000000e+00     547.559999
                      01              inf   27855.620429
                      02              inf    2449.940010
2018-06-11/2018-06-17 00     0.000000e+00     636.090000
                      01              inf   30393.230500
                      02              inf    1971.470007
2018-06-18/2018-06-24 00     0.000000e+00     961.240000
                      01              inf   28093.310436
                      02              inf    2401.470007
2018-06-25/2018-07-01 00     0.000000e+00     148.050000
                      01              inf    5864.300093
                      02    2.020202e+250     500.660001